In [1]:
import numpy as np
import pandas as pd
import os



In [13]:
# User inputs
T = float(input("Input Temp (C): "))
S = float(input("Input Salinity (g/kg): "))
L = float(input("Volume to be prepared (L): "))
Mg_multiple = float(input("Concentration multiplier for Mg: "))
Ca_multiple = float(input("Concentration multiplier for Ca: "))
SO4_multiple = float(input("Concentration multiplier for SO4: "))

#Molality (mol/Kg art.SW) of constituent ions in the ASW 
  # The Molality (mol/kg) value for ions from 
  # "Guide to best practices for ocean CO2 measurements"
m_Cl = (0.54922)/(35/S)
m_SO4 = (0.02824)/(35/S)
m_Na = (0.46911)/(35/S)
m_Mg = (0.05283)/(35/S)
m_Ca = (0.01036)/(35/S)
m_K = (0.01021)/(35/S)

m_B = (0.00042)/(35/S) #assuming all boron from one source

# m_CO32 = 0.001
m_HCO3 = 0.002



# Concentration of solutions (mol/L)
# MgCl2_conc = 2.009392
MgCl2_conc = 2.305002
CaCl2_conc = 1.094735

# Seawater density calculation
rhow = (
    999.842594 + 6.793952e-2 * T - 9.095290e-3 * T**2
    + 1.001685e-4 * T**3 - 1.120083e-6 * T**4 + 6.536332e-9 * T**5
)
A = 8.24493e-1 - 4.0899e-3 * T + 7.6438e-5 * T**2 - 8.2467e-7 * T**3 + 5.3875e-9 * T**4
B = -5.72466e-3 + 1.0227e-4 * T - 1.6546e-6 * T**2
C = 4.8314e-4

density = rhow + A * S + B * S**(3/2) + C * S**2
Density = density / 1000  # Convert to kg/L

# Moles of constituent ions
Cl = m_Cl * Density * L
SO4 = (m_SO4 * Density * L) * SO4_multiple
Na = (m_Na * Density * L) 
Mg = (m_Mg * Density * L) * Mg_multiple
Ca = (m_Ca * Density * L) * Ca_multiple
K = m_K * Density * L
B = m_B * Density * L

# CO32 = m_CO32 * Density * L
HCO3 = m_HCO3 * Density * L

# Molecular masses
Na2SO4_mw = 142.04
KCl_mw = 74.55
NaCl_mw = 58.44
BH3O3_mw = 61.83
Na2CO3_mw = 105.99
NaHCO3_mw = 84.01

# Masses to be added (g)
BH3O3 = B * BH3O3_mw
# Na2CO3 = CO32 * Na2CO3_mw
NaHCO3 = HCO3 * NaHCO3_mw 
Na2SO4 = SO4 * Na2SO4_mw
KCl = K * KCl_mw

# Solution volumes (mL)
MgCl2 = (Mg / MgCl2_conc) * 1000
CaCl2 = (Ca / CaCl2_conc) * 1000

Na_rest = Na - (2 * SO4 + HCO3)
Cl_rest = Cl - (K + 2 * Mg + 2 * Ca)

if Mg_multiple == 1 and Ca_multiple == 1 and SO4_multiple ==1:
    # Remaining Na and Cl to balance
    Na_rest = Na - (2 * SO4  + HCO3)
    NaCl = Na_rest * NaCl_mw

if Mg_multiple == 1 and Ca_multiple == 1 and SO4_multiple !=1:
    addedso4 = ((m_SO4 * Density * L) * SO4_multiple) - (m_SO4 * Density * L)
    NaCl = (Na_rest - (addedso4)) * NaCl_mw

if Mg_multiple!=1 and Ca_multiple==1 and SO4_multiple == 1 :
    addedmg = ((m_Mg * Density * L) * Mg_multiple) - (m_Mg * Density * L)
    NaCl = (Na_rest - (3 * addedmg)) * NaCl_mw

if Ca_multiple!=1 and Mg_multiple==1 and SO4_multiple ==1  :
    addedca = ((m_Ca * Density * L) * Ca_multiple) - (m_Ca * Density * L)
    NaCl = (Na_rest - (3 * addedca)) * NaCl_mw


# Display results
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(f"Total Volume to be prepared (L) = {L}")
print(f"Prpare at temp (°C) = {T}")
print(f"Concentration multiplier (Mg) = {Mg_multiple}")
print(f"Concentration multiplier (Ca) = {Ca_multiple}")
print(f"Concentration multiplier (SO4) = {SO4_multiple}")
print()
print(f"BH3O3 to be added (g) = {BH3O3}")
# print(f"Na2CO32 to be added (g) = {Na2CO3}")
print(f"NaHCO3 to be added (g) = {NaHCO3}")
print(f"KCl to be added (g) = {KCl}")
print(f"Na2SO4 to be added (g) = {Na2SO4}")
print(f"NaCl to be added (g) = {NaCl}")
print(f"MgCl2 solution to be added (mL) = {MgCl2}")
print(f"CaCl2 solution to be added (mL) = {CaCl2}")

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Total Volume to be prepared (L) = 1.0
Prpare at temp (°C) = 10.0
Concentration multiplier (Mg) = 1.0
Concentration multiplier (Ca) = 2.0
Concentration multiplier (SO4) = 1.0

BH3O3 to be added (g) = 0.026668516395906273
NaHCO3 to be added (g) = 0.17254854419722945
KCl to be added (g) = 0.7816704763284982
Na2SO4 to be added (g) = 4.119321372157785
NaCl to be added (g) = 22.77873080038933
MgCl2 solution to be added (mL) = 23.537461531439053
CaCl2 solution to be added (mL) = 19.437082004664457
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [15]:
#########################################################################################################

# ENTER MASSES FOR MEASURED SALTS IN GRAMS, AND MEASURED SOLUTIONS IN mL FOR PREPARED ARTIFICIAL SEAWATER

#########################################################################################################

#Measured Salts (g)
BH3O3 = float(input("BH3O3 (g): "))
# Na2CO3 = float(input("Na2CO3 (g): "))
NaHCO3 = float(input("NaHCO3 (g): "))
KCl = float(input("KCl (g): "))
Na2SO4 = float(input("Na2SO4 (g): "))
NaCl = float(input("NaCl (g): "))

#Measured solution (mL)
MgCl2 = float(input("MgCl2 (mL): "))
CaCl2 = float(input("CaCl2 (mL): "))

#Concentration of MgCl2 & CaCl2 solutions (mol/L) to be used
MgCl2_molarity = MgCl2_conc
CaCl2_molarity = CaCl2_conc 

mol_MgCl2 = MgCl2 * (MgCl2_molarity / 1000)
mol_CaCl2 = CaCl2 * (CaCl2_molarity / 1000)

#Ion Concentration
Mg_conc = ((mol_MgCl2))/(Density*L)
K_conc = ((KCl/KCl_mw))/(Density*L)
SO4_conc = ((Na2SO4/Na2SO4_mw))/(Density*L)
Ca_conc = ((mol_CaCl2))/(Density*L)
Na_conc = (((NaCl/NaCl_mw)+(2*Na2SO4/Na2SO4_mw)))/(Density*L)
Cl_conc = (((NaCl/NaCl_mw)+(2*mol_MgCl2)+(2*mol_CaCl2)+(KCl/KCl_mw)))/(Density*L)
Boron_conc = ((BH3O3/BH3O3_mw))/(Density*L)
# CO32_conc = (Na2CO3/Na2CO3_mw)/Density*L
HCO3_conc = (NaHCO3/NaHCO3_mw)/Density*L

salinity = (35.453 * Cl_conc) * 1.80655

water_correction = 0.9649125235

I = (0.5 * ((Cl_conc + (4 * SO4_conc) + Na_conc + (4 * Mg_conc) + (4 * Ca_conc) + K_conc)))/water_correction

print("Solution composition:")
print(f"Temperature (C) = {T}")
print(f"Ionic str (mol/kgH2O)= {I}")

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

water_correction = 1 - (S / 1000)  #for mol/kg soln to mol/kg h2o
print(f"[Na+] (mol/kg H2O) = {Na_conc / water_correction}")
print(f"[K+] (mol/kg H2O) = {K_conc / water_correction}")
print(f"[Mg2+] (mol/kg H2O) = {Mg_conc / water_correction}")
print(f"[Ca2+] (mol/kg H2O) = {Ca_conc / water_correction}")
print(f"[Cl-] (mol/kg H2O) = {Cl_conc / water_correction}")
print(f"[SO42-] (mol/kg H2O) = {SO4_conc / water_correction}")
# print(f"[CO32-] (mol/kg H2O) = {CO32_conc / water_correction}")
print(f"[HCO3-] (mol/kg H2O) = {HCO3_conc / water_correction}")
print(f"[Total Boron] (mol/kg H2O) = {Boron_conc / water_correction}")
print(f"[TA] (uEq/kg H2O) = {-10**6 * ((Na_conc + 2*Mg_conc + 2*Ca_conc + K_conc) - (+HCO3_conc + Cl_conc + 2*SO4_conc)) / water_correction}")


Solution composition:
Temperature (C) = 10.0
Ionic str (mol/kgH2O)= 0.7206326281820928
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[Na+] (mol/kg H2O) = 0.45203018621866753
[K+] (mol/kg H2O) = 0.010580710498867937
[Mg2+] (mol/kg H2O) = 0.054752018249491795
[Ca2+] (mol/kg H2O) = 0.02147472600375542
[Cl-] (mol/kg H2O) = 0.5565120373897334
[SO42-] (mol/kg H2O) = 0.029276173917148263
[HCO3-] (mol/kg H2O) = 0.0020707546455648535
[Total Boron] (mol/kg H2O) = 0.00043248295425414633
[TA] (uEq/kg H2O) = 2070.754645564936


In [62]:



Na_conc_corrected = Na_conc / water_correction
K_conc_corrected = K_conc / water_correction
Mg_conc_corrected = Mg_conc / water_correction
Ca_conc_corrected = Ca_conc / water_correction
Cl_conc_corrected = Cl_conc / water_correction
SO4_conc_corrected = SO4_conc / water_correction
CO32_conc_corrected = CO32_conc / water_correction
Boron_conc_corrected = Boron_conc / water_correction
TA_corrected = -10**6 * ((Na_conc + 2*Mg_conc + 2*Ca_conc + K_conc) - (2*CO32_conc + Cl_conc + 2*SO4_conc)) / water_correction


#Molalities
Na = Na_conc_corrected
K = K_conc_corrected
Mg = Mg_conc_corrected
Ca = Ca_conc_corrected
Cl = Cl_conc_corrected
SO4 = SO4_conc_corrected
CO32 = CO32_conc_corrected

TB= Boron_conc_corrected
DIC= CO32
TA = TA_corrected    

name = input("Enter file name (.csv): ")
# Create a dictionary with the data
data = {
    'Ion': ['Na+', 'K+', 'Mg2+', 'Ca2+', 'Cl-', 'SO42-', 'CO32-', 'Total Boron', 'TA'],
    'Concentration (mol/kg H2O)': [
        Na_conc_corrected, K_conc_corrected, Mg_conc_corrected, Ca_conc_corrected,
        Cl_conc_corrected, SO4_conc_corrected, CO32_conc_corrected, Boron_conc_corrected, TA_corrected
    ]
}

# Create a DataFrame
# Export the DataFrame to a CSV file
df = pd.DataFrame(data)
df.to_csv(f'{name}.csv', index=False)

# Get the current working directory
cwd = os.getcwd()

# Print the directory where the file has been exported
print(f"Data has been exported to {os.path.join(cwd, name + '.csv')}")


 

NameError: name 'pd' is not defined